In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import json
import urllib.request, json, requests
user_agent = {'User-agent': 'Mozilla/5.0'}
key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
alliances = []
for i in range(len(events)):
    print(i)
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/" + events[i] + "/alliances" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        temp = []
        temp.append(events[i])
        data = json.loads(url.read().decode())
        temp.append(data)
        alliances.append(temp)
        
        
        
        
        

In [ ]:
import urllib, requests, json
def get_year_matches(year):
    large_matches = []
    user_agent = {'User-agent': 'Mozilla/5.0'}
    key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/events/" +str(year) + "/keys" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        events = json.loads(url.read().decode())
        for event in events:
            req_match = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/" +event + "/matches/keys" + key, headers = user_agent)
            with urllib.request.urlopen(req_match) as url:
                matches = json.loads(url.read().decode())
                large_matches.append(matches)
    unrolled_matches = []
    for event in large_matches:
        for match in event:
            unrolled_matches.append(match)
    return unrolled_matches

In [ ]:
matches = get_year_matches(2019)

In [ ]:
#Gets every single team that has attended a comp this year/ Can also be used to 
#calculate how many tournaments a team took part in
def get_years_teams(year):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
    teams_big = []
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/events/" + str(year)+"/keys" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        events = json.loads(url.read().decode())
    for i in range(len(events)):
        req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/" + events[i] + "/teams/keys" + key, headers = user_agent)
        with urllib.request.urlopen(req) as url:
            teams = json.loads(url.read().decode())
            teams_big.append(teams)
    unrolled_teams = []
    for event in teams_big:
        for team in event:
            unrolled_teams.append(team)
    return unrolled_teams

In [ ]:
teams = get_years_teams(2019)

In [ ]:
#Gets alliance data
import requests
import urllib.request, json 
def get_alliances_year(year):
    large_alliances= []
    user_agent = {'User-agent': 'Mozilla/5.0'}
    key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/events/" + str(year)+"/keys" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        events = json.loads(url.read().decode())
    for i in range(len(events)):
        req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/" + events[i] + "/alliances" + key, headers = user_agent)
        with urllib.request.urlopen(req) as url:
            alliance_dict = json.loads(url.read().decode())
            alliance_selections = []
            try:
                for alliance in alliance_dict:
                    picks = alliance['picks']
                    alliance_selections.append(picks)
                large_alliances.append([events[i],alliance_selections])
            except:
                pass
    return large_alliances
            

In [ ]:
alliances = get_alliances_year(2019)

In [ ]:
def create_alliance_selection_dict(alliance_selections_big):
    d ={}
    res =[]
    for selections in alliance_selections_big:
        for j in range(8):
            try:
                for i in range(len(selections[1][j])):
                    arr = [selections[0],0,0,0,0,0]
                    arr[i+1] = j+1
                    if selections[1][j][i][3:] not in d.keys():
                        res.append(arr)
                        d[selections[1][j][i][3:]] = tuple(res)
                        res = []
                    else:
                        res = list(d[selections[1][j][i][3:]])
                        res.append(arr)
                        d[selections[1][j][i][3:]] = tuple(res)
                        res = []
            except:
                pass
    return d

In [ ]:
d = create_alliance_selection_dict(alliances)

In [ ]:
#Adds the dnp option by reasoning how many events they were in versus the number of alliances they were in
def add_dnps(dictionary,teams):
    for team in dictionary:
        if(team[-1]=="B"):
            team=team[:-1]
        temp = teams.count("frc"+str(team))-len(dictionary[team])
        arr = ['DNP',0,0,0,0,1]
        for i in range(temp):
            res = list(dictionary[team])
            res.append(arr)
            dictionary[team] = tuple(res)
    return dictionary

In [ ]:
d = add_dnps(d,teams)

In [ ]:
def get_match_data(matchid,event):
    match_data = []
    user_agent = {'User-agent': 'Mozilla/5.0'}
    key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/match/" + matchid + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        match = json.loads(url.read().decode())
    for i in range(2):
        if i ==0:
            color = 'blue'
        else:
            color = "red"
        teams = match['alliances'][color]['team_keys']
        score = match['alliances'][color]['score']
        if match['score_breakdown'][color]['bay1']=="PanelAndCargo":
            bay1h = 1
            bay1c = 1
        elif match['score_breakdown'][color]['bay1']=="Panel":
            bay1h = 1
            bay1c = 0
        else:
            bay1h = 0
            bay1c = 0
        if match['score_breakdown'][color]['bay2']=="PanelAndCargo":
            bay2h = 1
            bay2c = 1
        elif match['score_breakdown'][color]['bay2']=="Panel":
            bay2h = 1
            bay2c = 0
        else:
            bay2h = 0
            bay2c = 0
        if match['score_breakdown'][color]['bay3']=="PanelAndCargo":
            bay3h = 1
            bay3c = 1
        elif match['score_breakdown'][color]['bay3']=="Panel":
            bay3h = 1
            bay3c = 0
        else:
            bay3h = 0
            bay3c = 0
        if match['score_breakdown'][color]['bay4']=="PanelAndCargo":
            bay4h = 1
            bay4c = 1
        elif match['score_breakdown'][color]['bay4']=="Panel":
            bay4h = 1
            bay4c = 0
        else:
            bay4h = 0
            bay4c = 0
        if match['score_breakdown'][color]['bay5']=="PanelAndCargo":
            bay5h = 1
            bay5c = 1
        elif match['score_breakdown'][color]['bay5']=="Panel":
            bay5h = 1
            bay5c = 0
        else:
            bay5h = 0
            bay5c = 0
        if match['score_breakdown'][color]['bay6']=="PanelAndCargo":
            bay6h = 1
            bay6c = 1
        elif match['score_breakdown'][color]['bay6']=="Panel":
            bay6h = 1
            bay6c = 0
        else:
            bay6h = 0
            bay6c = 0
        if match['score_breakdown'][color]['bay7']=="PanelAndCargo":
            bay7h = 1
            bay7c = 1
        elif match['score_breakdown'][color]['bay7']=="Panel":
            bay7h = 1
            bay7c = 0
        else:
            bay7h = 0
            bay7c = 0
        if match['score_breakdown'][color]['bay8']=="PanelAndCargo":
            bay8h = 1
            bay8c = 1
        elif match['score_breakdown'][color]['bay8']=="Panel":
            bay8h = 1
            bay8c = 0
        else:
            bay8h = 0
            bay8c = 0
        auto_points = match['score_breakdown'][color]['sandStormBonusPoints']
        if match['score_breakdown'][color]['endgameRobot1'][0] == "H":
            robot1_climb = int(float(match['score_breakdown'][color]['endgameRobot1'][-1]))
        else:
            robot1_climb = 0
        if match['score_breakdown'][color]['endgameRobot2'][0] == "H":
            robot2_climb = int(float(match['score_breakdown'][color]['endgameRobot2'][-1]))
        else:
            robot2_climb = 0
        if match['score_breakdown'][color]['endgameRobot3'][0] == "H":
            robot3_climb = int(float(match['score_breakdown'][color]['endgameRobot3'][-1]))
        else:
            robot3_climb = 0

        if match['score_breakdown'][color]['preMatchBay1']=="Panel":
            bay1_preh = 1
            bay1_prec = 0
        else:
            bay1_preh = 0
            bay1_prec = 1
        if match['score_breakdown'][color]['preMatchBay2']=="Panel":
            bay2_preh = 1
            bay2_prec = 0
        else:
            bay2_preh = 0
            bay2_prec = 1
        if match['score_breakdown'][color]['preMatchBay3']=="Panel":
            bay3_preh = 1
            bay3_prec = 0
        else:
            bay3_preh = 0
            bay3_prec = 1
        if match['score_breakdown'][color]['preMatchBay6']=="Panel":
            bay6_preh = 1
            bay6_prec = 0
        else:
            bay6_preh = 0
            bay6_prec = 1
        if match['score_breakdown'][color]['preMatchBay7']=="Panel":
            bay7_preh = 1
            bay7_prec = 0
        else:
            bay7_preh = 0
            bay7_prec = 1
        if match['score_breakdown'][color]['preMatchBay8']=="Panel":
            bay8_preh = 1
            bay8_prec = 0
        else:
            bay8_preh = 0
            bay8_prec = 1
        try:
            robot1_hab_start = int(float(match['score_breakdown'][color]['preMatchLevelRobot1'][-1]))
        except:
            robot1_hab_start = 0
        try:
            robot2_hab_start = int(float(match['score_breakdown'][color]['preMatchLevelRobot2'][-1]))
        except:
            robot2_hab_start = 0
        try:
            robot3_hab_start = int(float(match['score_breakdown'][color]['preMatchLevelRobot3'][-1]))
        except:
            robot3_hab_start = 0

        if match['score_breakdown'][color]['lowLeftRocketFar'] == "PanelAndCargo":
            llrfh = 1
            llrfc = 1
        elif match['score_breakdown'][color]['lowLeftRocketFar'] == "Panel":
            llrfh = 1
            llrfc = 0
        else:
            llrfh = 0
            llrfc = 0
        if match['score_breakdown'][color]['lowLeftRocketNear'] == "PanelAndCargo":
            llrnh = 1
            llrnc = 1
        elif match['score_breakdown'][color]['lowLeftRocketNear'] == "Panel":
            llrnh = 1
            llrnc = 0
        else:
            llrnh = 0
            llrnc = 0
        if match['score_breakdown'][color]['lowRightRocketFar'] == "PanelAndCargo":
            lrrfh = 1
            lrrfc = 1
        elif match['score_breakdown'][color]['lowRightRocketFar'] == "Panel":
            lrrfh = 1
            lrrfc = 0
        else:
            lrrfh = 0
            lrrfc = 0
        if match['score_breakdown'][color]['lowRightRocketNear'] == "PanelAndCargo":
            lrrnh = 1
            lrrnc = 1
        elif match['score_breakdown'][color]['lowRightRocketNear'] == "Panel":
            lrrnh = 1
            lrrnc = 0
        else:
            lrrnh = 0
            lrrnc = 0

        if match['score_breakdown'][color]['midLeftRocketFar'] == "PanelAndCargo":
            mlrfh = 1
            mlrfc = 1
        elif match['score_breakdown'][color]['midLeftRocketFar'] == "Panel":
            mlrfh = 1
            mlrfc = 0
        else:
            mlrfh = 0
            mlrfc = 0
        if match['score_breakdown'][color]['midLeftRocketNear'] == "PanelAndCargo":
            mlrnh = 1
            mlrnc = 1
        elif match['score_breakdown'][color]['midLeftRocketNear'] == "Panel":
            mlrnh = 1
            mlrnc = 0
        else:
            mlrnh = 0
            mlrnc = 0
        if match['score_breakdown'][color]['midRightRocketFar'] == "PanelAndCargo":
            mrrfh = 1
            mrrfc = 1
        elif match['score_breakdown'][color]['midRightRocketFar'] == "Panel":
            mrrfh = 1
            mrrfc = 0
        else:
            mrrfh = 0
            mrrfc = 0
        if match['score_breakdown'][color]['midRightRocketNear'] == "PanelAndCargo":
            mrrnh = 1
            mrrnc = 1
        elif match['score_breakdown'][color]['midRightRocketNear'] == "Panel":
            mrrnh = 1
            mrrnc = 0
        else:
            mrrnh = 0
            mrrnc = 0

        if match['score_breakdown'][color]['topLeftRocketFar'] == "PanelAndCargo":
            tlrfh = 1
            tlrfc = 1
        elif match['score_breakdown'][color]['topLeftRocketFar'] == "Panel":
            tlrfh = 1
            tlrfc = 0
        else:
            tlrfh = 0
            tlrfc = 0
        if match['score_breakdown'][color]['topLeftRocketNear'] == "PanelAndCargo":
            tlrnh = 1
            tlrnc = 1
        elif match['score_breakdown'][color]['topLeftRocketNear'] == "Panel":
            tlrnh = 1
            tlrnc = 0
        else:
            tlrnh = 0
            tlrnc = 0
        if match['score_breakdown'][color]['topRightRocketFar'] == "PanelAndCargo":
            trrfh = 1
            trrfc = 1
        elif match['score_breakdown'][color]['topRightRocketFar'] == "Panel":
            trrfh = 1
            trrfc = 0
        else:
            trrfh = 0
            trrfc = 0
        if match['score_breakdown'][color]['topRightRocketNear'] == "PanelAndCargo":
            trrnh = 1
            trrnc = 1
        elif match['score_breakdown'][color]['topRightRocketNear']  == "Panel":
            trrnh = 1
            trrnc = 0
        else:
            trrnh = 0
            trrnc = 0

        match_data.append([event,[teams, score,robot1_hab_start,robot2_hab_start,robot3_hab_start,auto_points,bay1_preh, bay1_prec,bay2_preh,bay2_prec,bay3_preh,bay3_prec,
                      bay6_preh,bay6_prec,bay7_preh,bay7_prec,bay8_preh,bay8_prec,bay1h,bay1c,bay2h,bay2c,bay3h,bay3c,bay4h,bay4c,
                      bay5h,bay5c,bay6h,bay6c,bay7h,bay7c,bay8h,bay8c,
                     robot1_climb,robot2_climb,robot3_climb,llrfh,llrfc,llrnh,llrnc,lrrfh,lrrfc,lrrnh,lrrnc,mlrfh,mlrfc
                      ,mlrnh,mlrnc,mrrfh,mrrfc,mrrnh,mrrnc,tlrfh,tlrfc,tlrnh,tlrnc,trrfh,trrfc,trrnh,trrnc]])
    return match_data

In [ ]:
def get_matches_data(matches):
    arr =[]
    for match in matches:
        print(matches.index(match))
        try:
            arr.append(get_match_data(match,match.split("_")[0][4:]))
        except:
            pass
    return arr

In [ ]:
def get_all_match_data(year):
    matches = get_year_matches(2019)
    match_data = get_matches_data(matches)
    return match_data
        
        

In [ ]:
match_data = get_all_match_data(2019)

In [ ]:
alliance_selection_dict= {tuple([1,0,0,0,0]):420,tuple([2,0,0,0,0]):360,tuple([3,0,0,0,0]):300,tuple([4,0,0,0,0]):240,tuple([5,0,0,0,0]):200,tuple([6,0,0,0,0]):170,tuple([7,0,0,0,0]):150,tuple([8,0,0,0,0]):130,
                          tuple([0,1,0,0,0]):350,tuple([0,2,0,0,0]):290,tuple([0,3,0,0,0]):240,tuple([0,4,0,0,0]):200,tuple([0,5,0,0,0]):170,tuple([0,6,0,0,0]):150,tuple([0,7,0,0,0]):130,tuple([0,8,0,0,0]):120,
                          tuple([0,0,1,0,0]):70, tuple([0,0,2,0,0]):75, tuple([0,0,3,0,0]):80, tuple([0,0,4,0,0]):85, tuple([0,0,5,0,0]):90, tuple([0,0,6,0,0]):95, tuple([0,0,7,0,0]):100,tuple([0,0,8,0,0]):110,
                          tuple([0,0,0,1,0]):65, tuple([0,0,0,2,0]):65, tuple([0,0,0,3,0]):60, tuple([0,0,0,4,0]):60, tuple([0,0,0,5,0]):55, tuple([0,0,0,6,0]):55, tuple([0,0,0,7,0]):50, tuple([0,0,0,8,0]):50,
                          tuple([0,0,0,0,1]):10}

In [ ]:
alliance_selection_dict_defense = {tuple([1,0,0,0,0]):40,tuple([2,0,0,0,0]):40,tuple([3,0,0,0,0]):40,tuple([4,0,0,0,0]):40,tuple([5,0,0,0,0]):40,tuple([6,0,0,0,0]):40,tuple([7,0,0,0,0]):40,tuple([8,0,0,0,0]):40,
                                   tuple([0,1,0,0,0]):40,tuple([0,2,0,0,0]):40,tuple([0,3,0,0,0]):40,tuple([0,4,0,0,0]):40,tuple([0,5,0,0,0]):40,tuple([0,6,0,0,0]):80,tuple([0,7,0,0,0]):120,tuple([0,8,0,0,0]):160,
                                   tuple([0,0,1,0,0]):120, tuple([0,0,2,0,0]):130, tuple([0,0,3,0,0]):140, tuple([0,0,4,0,0]):150, tuple([0,0,5,0,0]):160, tuple([0,0,6,0,0]):170, tuple([0,0,7,0,0]):180,tuple([0,0,8,0,0]):190,
                                   tuple([0,0,0,1,0]):100, tuple([0,0,0,2,0]):100, tuple([0,0,0,3,0]):90, tuple([0,0,0,4,0]):90, tuple([0,0,0,5,0]):90, tuple([0,0,0,6,0]):80, tuple([0,0,0,7,0]):80, tuple([0,0,0,8,0]):80,
                                   tuple([0,0,0,0,1]):10}

In [ ]:
event_dict = {
    'caav':2,'azfl':3,'azpx':2,'arli':3,'lake':2,'tuis2':1,'ohcl':3,
    'bcvi':2,'abca':2,'ilpe':2,'mokc3':2,'nyut':2,'cafr':4,'code':3,
    'cadm':3,'qcmo':2,'qcqc':2,'nyro':3 ,'ndgf':3,'mokc':2,'paca':3,
    'hiho':2,'ksla':2,'nysu':2,'idbo':3,'iacf':3,'tuis':1,'mndu':3,
    'nvlv':2,'caln':3,'cala':3,'ohmv':3,'ilch':3,'mnmi':3,'mnmi2':3,
    'camb':3,'nyny':4,'nytr':2,'mndu2':3,'okok':3,'caoc':3,'flor':3,
    'scmb':3,'mxcm':1,'mxto':1,'mxmo':1,'alhu':2,'cada':4,'casd':3,
    'casf':4,'nyli2':2,'nyli':2,'wila':3,'casj':4,'tnkn':3,'flwp':3,
    'ausp':4,'ausc':3,'mosl':2,'utwv':3,'cave':3,'wimi':3,'mdbet':2,
    'vabla':2,'vahay':2,'mdowi':2,'mdoxo':2,'vapor':2,'vagle':2,'mialp':2,
    'mial2':2,'mibel':2,'micen':2,'midet':2,'miken':2,'mifor':2,'migib':2,
    'migul':2,'mijac':2,'miket':2,'mike2':2,'mikng':2,'milsu':2,'milak':2,
    'milan':2,'milin':2,'miliv':2,'mimar':2,'mimid':2,'mimil':2,'mimus':2,
    'mishe':2,'misou':2,'misjo':2,'mitvc':2,'mitry':2,'miwmi':2,'txama':2,
    'txaus':2,'txcha':2,'txdls':2,'txdel':2,'txelp':1,'txgre':2,'txpas':2,
    'txpla':2,'txsan':2,'paben':2,'njbri':2,'pahat':2,'njski':2,'njfla':2,
    'njtab':2,'paphi':2,'pawch':2,'ncgui':2,'ncash':2,'ncpem':2,'ncwak':2,
    'ingre':2,'inmis':2,'inwla':2,'isde1':2,'isde2':2,'isde3':2,'isde4':2,
    'macma':2,'nhgrs':2,'mabos':1,'cthar':2,'marea':2,'melew':2,'rismi':2,
    'mabri':2,'nhsnh':2,'nhdur':2,'ctwat':2,'mawne':2,'onosh':2,'onbar':2,
    'onto3':2,'onham':2,'onnob':2,'onto1':2,'onwat':2,'onlon':2,'onwin':2,
    'onnyo':2,'gaalb':2,'gacol':2,'gadal':2,'gafor':2,'gagai':2,'waahs':2,
    'waamv':2,'orore':2,'wasno':2,'orlak':2,'wamou':2,'wayak':2,'waspo':1,
    'orwil':2,'chchmp':3,'ftcmp':4,'iscmp':1,'mrcmp':4,'nccmp':2,'incmp':2,
    'necmp':4,'pncmp':4,'gacmp':2,'arc':5,'cars':5,'cur':5,
    'dal':5,'dar':5,'tes':5,'carv':5,'gal':5,'hop':5,'new':5,
    'roe':5,'tur':5,'cmpmi':5,'cmptx':5,'oncmp':3,'micmp':4,'oncmp1':2,
    'oncmp2':2,'micmp2':3,'micmp4':3,'micmp1':3,'micmp3':3, 'zhrcc':3,'cc':4,'caelk':2
    
}

In [ ]:
d['1072']

In [ ]:
#Formats the [event,1,0,0,0,0] list into ['event', score,'event',score] for defense
def convert_selections_defense(dictionary): 
    f = {}
    for team in dictionary:
        for item in dictionary[team]:
            try:
                f[item[0][4:]+" "+team] = alliance_selection_dict_defense[tuple(item[1:])]*event_dict[item[0][4:]]
            except:
                f[item[0][4:]+" "+team] = alliance_selection_dict_defense[tuple(item[1:])]
    return f
    

In [ ]:
#Formats the [event,1,0,0,0,0] list into ['event', score,'event',score] for defense
def convert_selections_offense(dictionary): 
    f = {}
    for team in dictionary:
        for item in dictionary[team]:
            try:
                f[item[0][4:]+" "+team] = alliance_selection_dict[tuple(item[1:])]*event_dict[item[0][4:]]
            except:
                f[item[0][4:]+" "+team] = alliance_selection_dict[tuple(item[1:])]
    return f
    

In [ ]:
f = convert_selections_offense(d)
g = convert_selections_defense(d)

In [ ]:
# Creates a dictionary for matches that take in a key['cafr 1072'] and output the match data vector
def convert_match_data_to_dict_offense(match_data):
    match_dict = {}
    res = []
    for match in match_data:
        for alliances in match:
            for team in alliances[1][0]:
                if alliances[0]+" " +team not in match_dict.keys():
                    index = alliances[1][0].index(team)
                    arr = []
                    arr.append(alliances[1][1])
                    arr.append(alliances[1][index+2])
                    for i in range(29):
                        arr.append(alliances[1][5+i])
                    arr.append(alliances[1][index+34])
                    for i in range(24):
                        arr.append(alliances[1][37+i])

                    res.append(arr) 
                    match_dict[alliances[0]+" "+team] = res
                    res = []
                else:
                    res = match_dict[alliances[0]+" "+team]
                    index = alliances[1][0].index(team)
                    arr = []
                    arr.append(alliances[1][1])
                    arr.append(alliances[1][index+2])
                    for i in range(29):
                        arr.append(alliances[1][5+i])
                    arr.append(alliances[1][index+34])
                    for i in range(24):
                        arr.append(alliances[1][37+i])
                    res.append(arr)
                    match_dict[alliances[0]+" "+team] = res
                    res = []
    return match_dict

In [ ]:
print(match_dict_defense['hop frc1072'][0][0][0])
#print(average_match_dict['hop frc4183'])


In [ ]:
# Creates a dictionary for matches that take in a key['cafr 1072'] and output the match data vector
# Doesn't work yet
def convert_match_data_to_dict_defense_part1(match_data):
    match_dict = {}
    res = []
    for match in match_data:
        for j in range(len(match)):
            for team in match[j][1][0]:
                if match[j][0]+" " +team not in match_dict.keys():
                    arr = []
                    arr.append(match[1-j][1])
                    res.append(arr) 
                    match_dict[match[j][0]+" "+team] = res
                    res = []
                else:
                    res = match_dict[match[j][0]+" "+team]
                    arr = []
                    arr.append(match[1-j][1])
                    res.append(arr)
                    match_dict[match[j][0]+" "+team] = res
                    res = []
    return match_dict

In [ ]:
match_dict_defense['hop frc1072'][0]

In [ ]:
def convert_defense_match_dict_to_average(match_dict):
    
    for key in match_dict:
        ave = np.zeros(56)
        match_datas = np.zeros(56)
        event = key.split(" ")[0]
        for match in match_dict[key]:
            teams = match[0][0]
            for team in teams:
                ave = ave+np.asarray(average_match_dict[event + " " + team])
            match_data = match[0][1:]
            for i in range(56):
                match_datas[i] = match_datas[i] + match_data[i]
            error = np.zeros(56)
            for i in range(len(ave)):
                ave[i] = ave[i] /3 /len(match_dict[key])
            for i in range(len(match_datas)):
                match_datas[i] = match_datas[i] / len(match_dict[key])
            error = error + ave-match_datas
        error = error/ len(match_dict[key])
        match_dict[key] = error
            
    return match_dict
                           
        

In [ ]:
'''def convert_defense_match_dict_to_average(match_dict_defense):
    res= []
    for key in match_dict_defense:
        team  = key.split(" ")[1]
        event = key.split(" ")[0]
        total_error = np.zeros(56)
        for match in match_dict_defense[key]:
            
            error = np.zeros(56)
            teams = match[0][0]
            ave = np.zeros(56)
            for i in range(3):
                for j in range(len(average_match_dict[event+" "+teams[i]])):
                    ave[j] = ave[j] + average_match_dict[event+" "+teams[i]][j]
            for i in range(len(arr)):
                ave[i] = ave[i]/3
            for i in range(3):
                error[0] = error[0]+ave[0]-match[0][1]
                error[1] = error[1]+ave[1]-match[0][2+i]
                for j in range(29):
                    error[j+2] = error[j+2] +ave[j+2]-match[0][5+j]
                error[31] = error[31]+ave[31]-match[0][i+34]
                for j in range(24):
                    error[j+32] = error[j+32] +ave[j+32]-match[0][37+j]
            for i in range(len(error)):
                error[i] = error[i]/3
            
        for i in range(len(error)):
            error[i] = error[i]/len(match_dict_defense[key])
        match_dict_defense[key] = error
    return match_dict_defense
            
                
   '''         

In [ ]:
match_dict_defense['cafr frc1072'][0]

In [ ]:
average_match_dict["cafr frc1072"]

In [ ]:
import numpy as np
#Averages the match values in the ccc match dictionary
def average_dict(match_dict):
    for key in match_dict:
        arr = np.zeros(56)
        for match in match_dict[key]:
            for i in range(56):
                arr[i] = arr[i] + match[i]
        for j in range(56):
            arr[j] = arr[j] / len(match_dict[key])
        match_dict[key] = arr
    return match_dict

In [ ]:
match_dict = convert_match_data_to_dict_offense(match_data)
match_dict_defense = convert_match_data_to_dict_defense_part1(match_data)


In [ ]:
average_match_dict = average_dict(match_dict)
match_dict_defense_averaged = convert_defense_match_dict_to_average(match_dict_defense)

In [ ]:
match_dict_defense_averaged['cafr frc1072']

In [ ]:
np.save("match dict defense averaged.npy", match_dict_defense_averaged)

In [ ]:
np.save("alliance selection dict for defense.npy",g)

In [ ]:
average_match_dict['hop frc1072']

In [ ]:
avergae_match_dict_defense['hop frc1072']

In [ ]:
user_agent = {'User-agent': 'Mozilla/5.0'}
key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
def add_oprs(average_match_dict):
    for i in list(average_match_dict):
        try:
            print(i)
            event = i.split(" ")[0]
            team = i.split(" ")[1][3:]
            req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/2019" +event+"/oprs" + key, headers = user_agent)
            with urllib.request.urlopen(req) as url:
                data = json.loads(url.read().decode())
            arr = list(average_match_dict[event+" frc"+team])
            arr.append(data['oprs']['frc'+team])
            #arr.append(data['dprs']['frc'+team])
            #arr.append(data['ccwms']['frc'+team])
            average_match_dict[i] = np.asarray(arr)
        except:
            del average_match_dict[i]
    return average_match_dict

In [ ]:
def get_opr(team,event):
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/2019" +event+"/oprs" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        data = json.loads(url.read().decode())
    try:
        return data['oprs'][team]
    except:
        return 0.0

In [ ]:
def add_average_event_data(average_match_dict_with_oprs):
    new_dict = {}
    for key in list(average_match_dict_with_oprs):
        try:
            event = key.split(" ")[0]
            team = key.split(" ")[1][3:]
            arr = np.asarray(list(average_match_dict_with_oprs[event+" frc"+team]))
            arr2 = np.asarray(get_average_data_event(event))
            print(arr2)
            temp = np.zeros(57)
            temp = arr-arr2
            new_arr = np.concatenate((arr,temp),axis = None)
            new_dict[key] = new_arr
            print(key)
            print(new_dict[key])
        except:
            del average_match_dict_with_oprs[key]
    return new_dict

In [ ]:
average_match_dict_with_oprs =  add_oprs(average_match_dict)


In [ ]:
import numpy as np
arr = np.asarray([2,3])
arr2 = np.asarray([4,5])
print(np.concatenate((arr, arr2), axis=None))

In [ ]:
np.save('temp_dict.npy',average_match_dict_with_oprs)

In [ ]:
average_match_dict_with_oprs = np.load('temp_dict.npy',allow_pickle = True).item()

In [ ]:
average_match_dict_with_oprs['caelk frc1072']

In [ ]:
average_match_dict_with_oprs_with_event_averages = add_average_event_data(average_match_dict_with_oprs)

In [ ]:
average_match_dict_defense_with_event_averages  = 

In [ ]:
len(average_match_dict_with_oprs_with_event_averages['cada frc1072'])

In [ ]:
np.save('average match dict(oprs and event averages).npy', average_match_dict_with_oprs_with_event_averages)

In [ ]:
average_match_dict_with_oprs_defense = add_oprs_dprs_ccwms(match_dict_defense_averaged)


In [ ]:
len(average_match_dict_with_oprs_defense['caelk frc1072'])

In [ ]:
np.save('alliance_dictionary(revised defense).npy',g)
np.save('match_dictionary(revised defense).npy',average_match_dict_with_oprs_defense)

In [ ]:
def get_average_data_event(eventid):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    key = '?X-TBA-Auth-Key=nkRpKyrwQU3P6XtznbFVHyq57mGz3N6e4kYUCwjigdBR6C50VYZ8FoKngBIySCiU'
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/2019" + eventid+"/insights" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        insights = json.loads(url.read().decode())
    qual_dict = insights['qual']
    average_score = qual_dict['average_score']
    average_hab_start = 1.666
    bay1_preh = qual_dict['average_cargo_ship_hatch_panel_preload_count'][0]/100
    bay1_prec = qual_dict['average_cargo_ship_cargo_preload_count'][0]/100
    bay2_preh = qual_dict['average_cargo_ship_hatch_panel_preload_count'][1]/100
    bay2_prec = qual_dict['average_cargo_ship_cargo_preload_count'][1]/100
    bay3_preh = qual_dict['average_cargo_ship_hatch_panel_preload_count'][2]/100
    bay3_prec = qual_dict['average_cargo_ship_cargo_preload_count'][2]/100
    bay6_preh = qual_dict['average_cargo_ship_hatch_panel_preload_count'][3]/100
    bay6_prec = qual_dict['average_cargo_ship_cargo_preload_count'][3]/100
    bay7_preh = qual_dict['average_cargo_ship_hatch_panel_preload_count'][4]/100
    bay7_prec = qual_dict['average_cargo_ship_cargo_preload_count'][4]/100
    bay8_preh = qual_dict['average_cargo_ship_hatch_panel_preload_count'][5]/100
    bay8_prec = qual_dict['average_cargo_ship_cargo_preload_count'][5]/100
    bay1h = qual_dict['average_cargo_ship_hatch_panel_count'][0]/100
    bay1c = qual_dict['average_cargo_ship_cargo_count'][0]/100
    bay2h = qual_dict['average_cargo_ship_hatch_panel_count'][1]/100
    bay2c = qual_dict['average_cargo_ship_cargo_count'][1]/100
    bay3h = qual_dict['average_cargo_ship_hatch_panel_count'][2]/100
    bay3c = qual_dict['average_cargo_ship_cargo_count'][2]/100
    bay4h = qual_dict['average_cargo_ship_hatch_panel_count'][3]/100
    bay4c = qual_dict['average_cargo_ship_cargo_count'][3]/100
    bay5h = qual_dict['average_cargo_ship_hatch_panel_count'][4]/100
    bay5c = qual_dict['average_cargo_ship_cargo_count'][4]/100
    bay6h = qual_dict['average_cargo_ship_hatch_panel_count'][5]/100
    bay6c = qual_dict['average_cargo_ship_cargo_count'][5]/100
    bay7h = qual_dict['average_cargo_ship_hatch_panel_count'][6]/100
    bay7c = qual_dict['average_cargo_ship_cargo_count'][6]/100
    bay8h = qual_dict['average_cargo_ship_hatch_panel_count'][7]/100
    bay8c = qual_dict['average_cargo_ship_cargo_count'][7]/100
    average_climb = (qual_dict['level1_climb_count'][2]
        +qual_dict['level2_climb_count'][2]*2
        +qual_dict['level3_climb_count'][2]*3)/qual_dict['level1_climb_count'][1]
    llrfh = qual_dict['average_rocket_hatch_panel_count']['lowLeftFar']/100
    llrfc = qual_dict['average_rocket_cargo_count']['lowLeftFar']/100
    llrnh = qual_dict['average_rocket_hatch_panel_count']['lowLeftNear']/100
    llrnc = qual_dict['average_rocket_cargo_count']['lowLeftNear']/100
    lrrfh = qual_dict['average_rocket_hatch_panel_count']['lowRightFar']/100
    lrrfc = qual_dict['average_rocket_cargo_count']['lowRightFar']/100
    lrrnh = qual_dict['average_rocket_hatch_panel_count']['lowRightNear']/100
    lrrnc = qual_dict['average_rocket_cargo_count']['lowRightNear']/100
    mlrfh = qual_dict['average_rocket_hatch_panel_count']['midLeftFar']/100
    mlrfc = qual_dict['average_rocket_cargo_count']['midLeftFar']/100
    mlrnh = qual_dict['average_rocket_hatch_panel_count']['midLeftNear']/100
    mlrnc = qual_dict['average_rocket_cargo_count']['midLeftNear']/100
    mrrfh = qual_dict['average_rocket_hatch_panel_count']['midRightFar']/100
    mrrfc = qual_dict['average_rocket_cargo_count']['midRightFar']/100
    mrrnh = qual_dict['average_rocket_hatch_panel_count']['midRightNear']/100
    mrrnc = qual_dict['average_rocket_cargo_count']['midRightNear']/100
    tlrfh = qual_dict['average_rocket_hatch_panel_count']['topLeftFar']/100
    tlrfc = qual_dict['average_rocket_cargo_count']['topLeftFar']/100
    tlrnh = qual_dict['average_rocket_hatch_panel_count']['topLeftNear']/100
    tlrnc = qual_dict['average_rocket_cargo_count']['topLeftNear']/100
    trrfh = qual_dict['average_rocket_hatch_panel_count']['topRightFar']/100
    trrfc = qual_dict['average_rocket_cargo_count']['topRightFar']/100
    trrnh = qual_dict['average_rocket_hatch_panel_count']['topRightNear']/100
    trrnc = qual_dict['average_rocket_cargo_count']['topRightNear']/100
    average_auton = qual_dict['average_sandstorm_bonus_auto']
    average_oprs = 0
    req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/2019" + eventid+"/oprs" + key, headers = user_agent)
    with urllib.request.urlopen(req) as url:
        dicts = json.loads(url.read().decode())
    opr_dict = dicts['oprs']
    for team in opr_dict:
        average_oprs = average_oprs + opr_dict[team]
    average_oprs = average_oprs/len(opr_dict)
    
    event_data = [average_score,average_hab_start,average_auton,bay1_preh,bay1_prec,bay2_preh,bay2_prec,bay3_preh,bay3_prec,
                 bay6_preh,bay6_prec,bay7_preh,bay7_prec,bay8_preh,bay8_prec,bay1h,bay1c,bay2h,bay2c,bay3h,bay3c,bay4h,bay4c,
                 bay5h,bay5c,bay6h,bay6c,bay7h,bay7c,bay8h,bay8c,average_climb,llrfh,llrfc,llrnh,llrnc,lrrfh,lrrfc,lrrnh,lrrnc,mlrfh,mlrfc
                      ,mlrnh,mlrnc,mrrfh,mrrfc,mrrnh,mrrnc,tlrfh,tlrfc,tlrnh,tlrnc,trrfh,trrfc,trrnh,trrnc,average_oprs]
    return event_data
    
    
    
    

In [ ]:
get_average_data_event('abca')

In [ ]:
matches = []
req = urllib.request.Request("https://www.thebluealliance.com/api/v3/event/2019caelk/matches/keys" + key, headers = user_agent)
with urllib.request.urlopen(req) as url:
    matchids = json.loads(url.read().decode())
ccc_match_data = get_matches_data(matchids)

In [ ]:
ccc_match_dict = convert_match_data_to_dict_offense(ccc_match_data)
ccc_match_dict_averaged  = average_dict(ccc_match_dict)
ccc_match_dict_averaged_with_oprs = add_oprs(ccc_match_dict_averaged)
ccc_match_dict_with_average_event_data = add_average_event_data_for_comp(ccc_match_dict_averaged_with_oprs)
#print(ccc_match_dict_with_average_event_data)

In [ ]:
ccc_match_dict_with_average_event_data['caelk frc1072']

In [ ]:
np.save('ccc_match_dict(large).npy',ccc_match_dict_with_average_event_data)

In [ ]:
ccc_match_data[0]

In [ ]:
def add_average_event_data_for_comp(ccc_match_dict_averaged_with_oprs):
    new_dict = {}
    new_arr = np.zeros(57)
    for key in list(ccc_match_dict_averaged_with_oprs):
        new_arr  = new_arr + np.asarray(ccc_match_dict_averaged_with_oprs[key])
    new_arr = new_arr/len(ccc_match_dict_averaged_with_oprs)
    for key in list(ccc_match_dict_averaged_with_oprs):
        new_dict[key] = np.concatenate((ccc_match_dict_averaged_with_oprs[key], ccc_match_dict_averaged_with_oprs[key]-new_arr), axis = None)
    return new_dict


